In [233]:
# Overleaf
# https://www.overleaf.com/9836375457csrqydskhcmj

# Init

In [234]:
# Libraries
import numpy as np
from scipy.stats import norm

In [235]:
# Option Characteristics
S0 = 100
K = 99
r = 0.06
sigma = 0.2
T = 1.0
M = 10_000
np.random.seed = 42

# Monte Carlo

In [236]:

def mc_payoff(results, call, payoff_type):
    # Find option payoffs
    if payoff_type == "vanilla":
        if call:
            V = np.maximum(results - K, 0)
        else:
            V = np.maximum(K - results, 0)
    if payoff_type == "digital":
        if call:
            V = np.maximum(results - K, 0)
            V[V != 0] = 1
        else:
            V = np.maximum(K - results, 0)
            V[V != 0] = 1
    return V



# It uses GBM for underlying, we might need to use another scheme from provided papers, not sure
def monte_carlo_option_valuations(S0, K, r, sigma, T, M, call=True, payoff="vanilla"):
    """
    N - Amount of Runs in a Simulation
    M - Amount of Simulations
    """

    dt = T/1    # N=1 Because of Geometric Brownian-Motion we can just simulate the variables at the final Time Step as Brownian Motion scales with time and independent increments.
    results = np.zeros(M)
    
    # Monte-Carlo Runs
    for i in range(M):
        S = S0 * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * np.random.normal())
        results[i] = S
    
    # Find option payoffs
    V = mc_payoff(results, call, payoff)
 
    return V


# Finding Fair Value from Monte Carlo Simulation
mc_results = monte_carlo_option_valuations(S0, K, r, sigma, T, M, True, "vanilla")
option_price = np.exp(-r * T) * np.mean(mc_results)     # We have to discount the payoffs to calculate fair option's price
print(f"Option price: {option_price:.4f}")

# Finding Confidence Interval
standard_error = np.std(mc_results, ddof=1) / np.sqrt(len(mc_results))
t_stat = np.abs(norm.ppf(0.025))
margin_of_error = t_stat * standard_error
lower_bound = option_price - margin_of_error
upper_bound = option_price + margin_of_error
print(f"Standard error: {standard_error:.4f}")
print(f"95% Confidence interval: ({lower_bound:.4f}, {upper_bound:.4f})")

Option price: 0.5662
Standard error: 0.0049
95% Confidence interval: (0.5566, 0.5758)


# Approximating Delta

In [237]:
# Finite Difference Method

# Finding New Prices
finite_difference_step = 0.1       # Not sure how to optimize this
delta_S = S0 * finite_difference_step
S_up = S0 + delta_S
S_down = S0 - delta_S
# Finding Approxiamation for Higher Price
mc_results_up = monte_carlo_option_valuations(S_up, K, r, sigma, T, M, True)
payoff_up = np.exp(-r * T) * np.mean(mc_results_up)
# Finding Approxiamation for Lower Price
mc_results_down = monte_carlo_option_valuations(S_down, K, r, sigma, T, M, True)
payoff_down = np.exp(-r * T) * np.mean(mc_results_down)
# Approximation of Delta with Finite Differences
delta_approx = (np.mean(payoff_up) - np.mean(payoff_down)) / (2 * delta_S)

# Print the results
print(f"Delta approximation: {delta_approx:.4f}")

Delta approximation: 0.6525
